In [ ]:
# Justas Baniulis;LSP: 2015956; Klases: Parachute, Mushroom, Pizza; Modelis: VGG19

In [2]:
import torch
from torchvision import models
import numpy as np

In [15]:
# GPU ir modelio atsisiuntimas
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
weights = models.VGG19_Weights.IMAGENET1K_V1
model = models.vgg19(weights=weights).to(device)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [4]:
!pip install openimages

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.4 MB/s eta 0:00:00


In [6]:
import os
from openimages.download import download_dataset
import glob
import shutil


# atsisiusti duomenis i direktorija
data_dir = "data"

# pravalymui direktorijos
#shutil.rmtree(data_dir)

# Pasirenkam klases ir ju kieki
chosen_classes = [('Parachute', 360), ('Mushroom', 360), ('Pizza', 360)]

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

for name, count in chosen_classes:
  print(name)
  download_dataset(data_dir, class_labels=[name], limit=count)

Parachute


100%|██████████| 360/360 [00:05<00:00, 68.22it/s]


Mushroom


100%|██████████| 3/3 [00:00<00:00,  7.36it/s]


Pizza


100%|██████████| 360/360 [00:05<00:00, 66.74it/s]


In [8]:
from PIL import Image
# atsikratom greyscaled nuotrauku kiekvienos klases
for c_temp in chosen_classes:
  directory = data_dir + "/{}/images".format(c_temp[0].lower())
  for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    with Image.open(filepath) as img:
      if img.mode == "L":
        # If the image is greyscale, delete it
        os.remove(filepath)
        print(f"Deleted {filepath} because it is greyscale.")

Deleted data/parachute/images/52c6153ed8637197.jpg because it is greyscale.
Deleted data/parachute/images/4f9f1a3167a175e4.jpg because it is greyscale.
Deleted data/parachute/images/5312e8dabd0f5346.jpg because it is greyscale.
Deleted data/parachute/images/0c7c13bc5d7b5e16.jpg because it is greyscale.
Deleted data/parachute/images/11225278874edc90.jpg because it is greyscale.
Deleted data/parachute/images/2435c841e5ab4c0a.jpg because it is greyscale.
Deleted data/pizza/images/39c8b88543a2e207.jpg because it is greyscale.


In [9]:

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from PIL import Image

# Transformacijos
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset nuotraukai
class ImageDataset(Dataset):
    def __init__(self, images_dir):
        # direktorija ir transformacijos dataset'ui
        self.images_dir = images_dir
        self.transform = transform

        # failu path'ai. (failas, klase)
        self.class1_files = [(item, chosen_classes[0][0]) for item in glob.glob(self.images_dir + "/{}/images/*.jpg".format(chosen_classes[0][0].lower()))]
        self.class2_files = [(item, chosen_classes[1][0]) for item in glob.glob(self.images_dir + "/{}/images/*.jpg".format(chosen_classes[1][0].lower()))]
        self.class3_files = [(item, chosen_classes[2][0]) for item in glob.glob(self.images_dir + "/{}/images/*.jpg".format(chosen_classes[2][0].lower()))]

        # visi failai
        self.files = self.class1_files + self.class2_files + self.class3_files
         
    def __len__(self):
        return (len(self.files))
    
    # grazinsim faila ir jo klase
    def __getitem__(self, i):
        file_i = self.files[i]
        
        im = Image.open(file_i[0])

        if self.transform:
          try:
            image = self.transform(im)
          except RuntimeError:
            raise Exception(f"Faulty image: '{file_i[0]}'")
            
        y = file_i[1]
        return image, y


In [10]:
def calculate_metrics(calculations):
  
  TP = calculations['TP']
  TN = calculations['TN']
  FP = calculations['FP']
  FN = calculations['FN']

  metrics = {}
  metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
  metrics['recall'] = TP / (TP + FN)
  metrics['precision'] = TP / (TP + FP)
  metrics['F1'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])

  return metrics

In [16]:
# rinkinys, darbininkai, slenkstine reiksme
batch_n = 30
workers_n = 4
threshold = 0.05

# duomenu aibe ir paruosejas
dataSet = ImageDataset(data_dir)
dataLoader = DataLoader(dataSet, batch_size=batch_n, shuffle=True)

# pasiruosiam sarasa su klasem (id, vardas, {TP, TN, FP, FN})
classes_calc = []
for class_i in chosen_classes:
  calc = {'TP':0, 'TN':0, 'FP':0, 'FN':0}
  classes_calc.append((weights.meta["categories"].index(class_i[0].lower()),class_i[0], calc))

# dedam rinkinius i modeli ir skaiciuojam TP, TN, FP, FN
for batch, class_names in dataLoader:
  batch = batch.to(device)
  predictions = model(batch).softmax(0)

  #iteruojam per rinkini spejimu
  for i in range(0, len(predictions)):
    prediction = predictions[i]

    #iteruojam per pasirinktas klases
    for c in classes_calc:

      # jeigu perzerngiam slenksti, tai esam positive
      if prediction[c[0]] >= threshold:

        if(c[1] == class_names[i]):
          c[2]['TP'] += 1

        # type 1 error
        else:
          c[2]['FP'] += 1

      # jeigu NEperzerngiam slenksti, tai esam negative    
      else:

        # type 2 error
        if(c[1] == class_names[i]):
          c[2]['FN'] += 1

        else:
          c[2]['TN'] += 1


In [22]:
# rezultatai:
TP_t= 0
TN_t= 0
FP_t= 0
FN_t = 0

for c in classes_calc:
  print(c[1])
  print(calculate_metrics(c[2]))
  print("   ")

  TP_t += c[2]['TP']
  TN_t += c[2]['TN']
  FP_t += c[2]['FP']
  FN_t += c[2]['FN']

all_metrics = calculate_metrics({'TP': TP_t, 'TN': TN_t, 'FP': FP_t, 'FN': FN_t})
print("All")
print(all_metrics)

Parachute
{'accuracy': 0.7194780987884436, 'recall': 0.1497175141242938, 'precision': 1.0, 'F1': 0.2604422604422605}
   
Mushroom
{'accuracy': 0.7399813606710158, 'recall': 0.225, 'precision': 1.0, 'F1': 0.36734693877551017}
   
Pizza
{'accuracy': 0.7287977632805219, 'recall': 0.1894150417827298, 'precision': 1.0, 'F1': 0.3185011709601873}
   
All
{'accuracy': 0.7294190742466604, 'recall': 0.18825722273998136, 'precision': 1.0, 'F1': 0.31686274509803924}
